In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.3'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# View schema
home_sales_df.printSchema()

# Set data types
home_sales_df = home_sales_df.withColumn('price', home_sales_df['price'].cast('float'))
home_sales_df = home_sales_df.withColumn('date', home_sales_df['date'].cast('date'))
home_sales_df = home_sales_df.withColumn('date_built', home_sales_df['date_built'].cast('date'))
home_sales_df = home_sales_df.withColumn('sqft_living', home_sales_df['sqft_living'].cast('int'))
home_sales_df = home_sales_df.withColumn('view', home_sales_df['view'].cast('int'))

# View updated schema
home_sales_df.printSchema()

# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('sales_view')

# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
select
 year(date)
,round(avg(price),2)
from sales_view
where bedrooms = '4'
group by 1
order by 1 desc
""").show()


# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
select
 year(date_built)
,round(avg(price),2)
from sales_view
where bedrooms = '3'
and bathrooms = '3'
group by 1
order by 1 desc
""").show()

# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
select
 year(date_built)
,round(avg(price),2)
from sales_view
where bedrooms = '3'
and bathrooms = '3'
and floors = '2'
and sqft_living >= 2000
group by 1
order by 1 desc
""").show()

# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""
select
 view
,round(avg(price),2)
from sales_view
where price >= 350000
group by 1
order by 1 desc
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

# 8. Check if the table is cached.
spark.catalog.isCached('sales_view')

# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
spark.sql("""
select
 view
,round(avg(price),2)
from sales_view
where price >= 350000
group by 1
order by 1 desc
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy('date_built').mode('overwrite').parquet('home_sales_partitioned')

# 11. Read the parquet formatted data.
p_home_sales_p=spark.read.parquet('home_sales_partitioned')

# 12. Create a temporary table for the parquet data.
p_home_sales_p.createOrReplaceTempView('p_sales_p')

# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

start_time = time.time()
spark.sql("""
select
 view
,round(avg(price),2)
from p_sales_p
where price >= 350000
group by 1
order by 1 desc
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales_view")

# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("sales_view"):
  print("a table is till cached")
else:
  print("all clear")



Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
